In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import spacy

!python -m spacy download pt_core_news_lg
pln = spacy.load('pt_core_news_lg')
pln

from spacy.matcher import Matcher

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
caminho_arquivo = '/content/drive/My Drive/Dataset/dataset.csv'

dataframe = pd.read_csv(caminho_arquivo)
dataframe.head()

,Nome da Música,Artista,Gênero Musical,Letra da Música
0,Carolina,Seu Jorge,MPB,Carolina é uma menina bem difícil de esquecer ...
1,Epitáfio,Titãs,Rock,Devia ter amado mais Ter chorado mais Ter vist...
2,Lugar Ao Sol,Charlie Brown Jr.,Rock,"Que bom viver, como é bom sonhar E o que ficou..."
3,Relicário - Ao Vivo,Cássia Eller,Rock,É uma índia com colar A tarde linda que não qu...
4,Você Me Vira A Cabeça (Me Tira Do Sério),Alcione,Samba,"Você me vira a cabeça, me tira do sério Destró..."


In [ ]:
import re

def limpar_texto(musica):
    if isinstance(musica, str):
        musica = re.sub(r'\(.*?\)', '', musica)  # Remove tudo que está entre parênteses
        musica = re.sub(r'\[.*?\]', '', musica)  # Remove tudo que está entre colchetes
        musica = re.sub(r'\n+', ' ', musica)  # Remove os caracteres "\n"
        musica = re.sub(r'\s+', ' ', musica).strip()  # Remove espaços extras
    return musica

dataframe['Letra da Música'] = dataframe['Letra da Música'].apply(limpar_texto)

In [ ]:
sujeitos_femininos = ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas", "novinha", "novinhas", "mina", "minas"]
sujeitos_masculinos = ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "rapaz", "rapazes", "moço", "moços", "cavalheiro", "cavalheiros", "rei", "reis", "marido", "maridos", "namorado", "namorados", "novinho", "novinhos"]

In [ ]:
def carregar_palavras(nome_arquivo):
    with open(nome_arquivo, 'r') as arquivo:
        conteudo = arquivo.read()
        palavras = conteudo.split()
        return palavras

nome_arquivo = '/content/drive/MyDrive/TCC - UFMS/BRACIS/Profissões/Lista de Profissões/profissoes_masc.txt'
palavras_procuradas = carregar_palavras(nome_arquivo)
print(palavras_procuradas)

nome_arquivo = '/content/drive/MyDrive/TCC - UFMS/BRACIS/Profissões/Lista de Profissões/profissoes_casos_especiais.txt'
palavras_neutras = carregar_palavras(nome_arquivo)
print(palavras_neutras)

profissoes = palavras_procuradas + palavras_neutras

['legislador', 'ministro', 'diretor', 'professor', 'engenheiro', 'desenvolvedor', 'programador', 'agrônomo', 'arquiteto', 'biólogo', 'médico', 'veterinário', 'farmacêutico', 'enfermeiro', 'técnico', 'advogado', 'procurador', 'governador', 'vereador', 'prefeito', 'deputado', 'senador', 'defensor', 'juíz', 'promotor', 'delegado', 'psicólogo', 'pesquisador', 'historiador', 'filósofo', 'auditor', 'administrador', 'contador', 'secretário', 'redator', 'tradutor', 'escritor', 'locutor', 'produtor', 'coreógrafo', 'ator', 'compositor', 'desenhista', 'decorador', 'bailarino', 'músico', 'cantor', 'escultor', 'pintor', 'floricultor', 'inspetor', 'corretor', 'leiloeiro', 'avaliador', 'bibliotecário', 'fotógrafo', 'apresentador', 'dançarino', 'escrivão', 'entrevistador', 'mordomo', 'cozinheiro', 'camareiro', 'garçom', 'barman', 'copeiro', 'astrólogo', 'bombeiro', 'lavadeiro', 'repositor', 'instalador', 'pescador', 'caçador', 'montador', 'joalheiro', 'vidreiro', 'ceramista', 'artesão', 'ourives', 'ma

#Extraindo profissoes no Masculino - expressao regular - ajustado para profissoes

In [ ]:
matcher2 = Matcher(vocab=pln.vocab)

#Você + Auxiliar + Adjetivo (Masculino)
pronoun_aux_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito + Auxiliar + Adjetivo
sujeito_aux_adj = [{'LOWER': {'IN': sujeitos_masculinos}}, {'POS': 'AUX'}, {"LOWER": {"IN": profissoes}}]
#Você + Auxiliar + muito/pouco + Adjetivo
pronoun_aux_adv_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito + Auxiliar + muito/pouco + Adjetivo
sujeito_aux_adv_adj = [{'LOWER': {'IN':sujeitos_masculinos}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {"LOWER": {"IN": profissoes}}]  # Ela é muito [pouco] inteligente.
#Voce + Auxiliar + Auxiliar + Adjetivo (Masculino)
pronoun_aux_aux_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito + Auxiliar + Auxiliar + Adjetivo
sujeito_aux_aux_adj = [{'LOWER': {'IN': sujeitos_masculinos}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {"LOWER": {"IN": profissoes}}]
#Voce + Auxiliar + Auxiliar + Muito/pouco + Adjetivo (Masculino)
pronoun_aux_aux_adv_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito + Auxiliar + Auxiliar + muito/pouco + Adjetivo
sujeito_aux_aux_adv_adj = [{'LOWER': {'IN': sujeitos_masculinos}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#Sujeito + Auxiliar + Verbo + Adjetivo
sujeito_aux_verb_adj = [{'LOWER': {'IN': sujeitos_masculinos}}, {'POS': 'AUX'}, {"POS":"VERB"}, {"LOWER": {"IN": profissoes}}]
#Sujeito + Auxiliar + Verbo + muito/pouco + Adjetivo
sujeito_aux_verb_adv_adj = [{'LOWER': {'IN':sujeitos_masculinos}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#Voce + Auxiliar + Verbo + Adjetivo (Masculino)
pronoun_aux_verb_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {"POS":"VERB", "LOWER":"sendo"}, {"LOWER": {"IN": palavras_procuradas}}]
#Voce + Auxiliar + Verbo + muito/pouco + Adjetivo (Masculino)
pronoun_aux_verb_adv_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {"POS":"VERB", "LOWER":"sendo"}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito + Verbo + Auxiliar + Adjetivo
sujeito_verb_aux_adj = [{'LOWER': {'IN': sujeitos_masculinos}}, {'POS': 'VERB'}, {'POS': 'AUX', "LOWER":"sendo"}, {"LOWER": {"IN": profissoes}}]
#Sujeito + Verbo + Auxiliar + muito/pouco + Adjetivo
sujeito_verb_aux_adv_adj = [{'LOWER': {'IN': sujeitos_masculinos}}, {'POS': 'VERB'}, {'POS': 'AUX', "LOWER":"sendo"}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#Você + Verbo + Auxiliar + Adjetivo (Masculino)
pronoun_verb_aux_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {"LOWER": {"IN": palavras_procuradas}}]
#Você + Verbo + Auxiliar + muito/pouco + Adjetivo (Masculino)
pronoun_verb_aux_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]

#Sujeito + Verbo + Adjetivo / LEMMA tornar, ficar, parecer
sujeito_verb_adj = [{'LOWER': {'IN':sujeitos_masculinos}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {"LOWER": {"IN": profissoes}}]
#Sujeito + Verbo + muito/pouco + Adjetivo / LEMMA tornar, ficar, parecer
sujeito_verb_adv_adj = [{'LOWER': {'IN':sujeitos_masculinos}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#Voce + Verbo + Adjetivo / LEMMA tornar, ficar, parecer
pronoun_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {"LOWER": {"IN": palavras_procuradas}}]
#Voce + Verbo + muito/pouco + Adjetivo / LEMMA tornar, ficar, parecer
pronoun_verb_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer","tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito + Adjetivo
sujeito_adj = [{'LOWER': {'IN': sujeitos_masculinos}}, {"LOWER": {"IN": profissoes}}]
#Sujeito + Pronome +  Verbo (LEMMA) + Adjetivo / Ela se tornou bonita
sujeito_pron_verb_adj = [{'LOWER': {'IN':sujeitos_masculinos}}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {"LOWER": {"IN": profissoes}}]
#Você +Pronome +  Verbo (LEMMA) + Adjetivo / Você se tornou bonita
pronoun_pron_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'PRON', "LOWER": "se"},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito +Pronome +  Verbo (LEMMA) + Adjetivo / Ela se tornou muito bonita
sujeito_pron_verb_adv_adj = [{'LOWER':  {'IN':sujeitos_masculinos}}, {'POS':'PRON', "LOWER": "se"}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#Você +Pronome +  Verbo (LEMMA) + Adjetivo / Você se tornou muito bonita
pronoun_pron_verb_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito + Auxiliar + Pronome +  Verbo (LEMMA) + Adjetivo
sujeito_aux_pron_verb_adj = [{'LOWER': {'IN':sujeitos_masculinos}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {"LOWER": {"IN": profissoes}}]
#Voce + Auxiliar + Pronome +  Verbo (LEMMA) + Adjetivo
pronoun_aux_pron_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer","tornar"]}}, {"LOWER": {"IN": palavras_procuradas}}]
#Voce + Auxiliar + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
pronoun_aux_pron_verb_adv_adj = [{'POS': 'PRON',"MORPH":{"IN":['Case=Nom|Number=Sing|Person=3|PronType=Prs']}},{'POS':'AUX'}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer",  "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito + Auxiliar + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
sujeito_aux_pron_verb_adv_adj = [{'LOWER': {'IN': sujeitos_masculinos}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]
#Sujeito + Verbo + Pronome + Verbo (LEMMA) + Adjetivo
sujeito_verb_pron_verb_adj = [{'LOWER': {'IN':sujeitos_masculinos}}, {'POS': "VERB"}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer",  "tornar"]}}, {"LOWER": {"IN": palavras_procuradas}}]
# Você + Verbo + Pronome +  Verbo (LEMMA) + Adjetivo
pronoun_verb_pron_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'VERB'}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {"LOWER": {"IN": palavras_procuradas}}]
# Sujeito + Verbo + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
sujeito_verb_pron_verb_adv_adj = [{'LOWER': {'IN': sujeitos_masculinos}}, {'POS': "VERB"}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer",  "tornar"]}},{'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]
# Você + Verbo + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
pronoun_verb_pron_verb_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'VERB'}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer",  "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": palavras_procuradas}}]
# Nome proprio + aux + adj
nome_aux_adj = [{"ENT_TYPE": "PER", 'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'AUX'}, {"LOWER": {"IN": profissoes}}]
#nome + Auxiliar + muito/pouco + Adjetivo
nome_aux_adv_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#nome + Auxiliar + Auxiliar + Adjetivo
nome_aux_aux_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {"LOWER": {"IN": profissoes}}]
#nome + Auxiliar + Auxiliar + muito/pouco + Adjetivo
nome_aux_aux_adv_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#nome + Auxiliar + Verbo + Adjetivo
nome_aux_verb_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'AUX'}, {"POS":"VERB"}, {"LOWER": {"IN": profissoes}}]
#nome + Auxiliar + Verbo + muito/pouco + Adjetivo
nome_aux_verb_adv_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#nome + Verbo + Auxiliar + Adjetivo
nome_verb_aux_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {"LOWER": {"IN": profissoes}}]
#nome + Verbo + Auxiliar + muito/pouco + Adjetivo
nome_verb_aux_adv_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#nome + Auxiliar(Ser/Estar) + verbo
nome_aux_verb = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'AUX'}, {"LOWER": {"IN": profissoes}}]

#nome + Verbo + Adjetivo / LEMMA tornar, ficar, parecer
nome_verb_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {"LOWER": {"IN": profissoes}}]
#nome + Verbo + muito/pouco + Adjetivo / LEMMA tornar, ficar, parecer
nome_verb_adv_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#nome + Adjetivo
nome_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {"LOWER": {"IN": profissoes}}]
#nome + Pronome +  Verbo (LEMMA) + Adjetivo / Ela se tornou bonita
nome_pron_verb_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {"LOWER": {"IN": profissoes}}]
#nome +Pronome +  Verbo (LEMMA) + Adjetivo / Ela se tornou muito bonita
nome_pron_verb_adv_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#nome + Auxiliar + Pronome +  Verbo (LEMMA) + Adjetivo
nome_aux_pron_verb_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {"LOWER": {"IN": profissoes}}]
#nome + Auxiliar + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
nome_aux_pron_verb_adv_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]
#nome + Verbo + Pronome + Verbo (LEMMA) + Adjetivo
nome_verb_pron_verb_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': "VERB"}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer",  "tornar"]}}, {"LOWER": {"IN": profissoes}}]
# nome + Verbo + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
nome_verb_pron_verb_adv_adj = [{"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Masc|Number=Sing']}}, {'POS': "VERB"}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer",  "tornar"]}},{'POS': "ADV", 'OP': '+'}, {"LOWER": {"IN": profissoes}}]


# Adicionando os padrões
matcher2.add('pronoun+aux+adj', patterns=[pronoun_aux_adj])
matcher2.add('sujeito_aux_adj', patterns=[sujeito_aux_adj])
matcher2.add('pronoun+aux+adv+adj', patterns=[pronoun_aux_adv_adj])
matcher2.add('sujeito_aux_adv_adj', patterns=[sujeito_aux_adv_adj])
matcher2.add('pronoun_aux_aux_adj ', patterns=[pronoun_aux_aux_adj])
matcher2.add('sujeito_aux_aux_adj', patterns=[sujeito_aux_aux_adj])
matcher2.add('pronoun_aux_aux_adv_adj', patterns=[pronoun_aux_aux_adv_adj])
matcher2.add('sujeito_aux_aux_adv_adj', patterns=[sujeito_aux_aux_adv_adj])
matcher2.add('sujeito_aux_verb_adj', patterns=[sujeito_aux_verb_adj])
matcher2.add('sujeito_aux_verb_adv_adj', patterns=[sujeito_aux_verb_adv_adj])
matcher2.add('pronoun_aux_verb_adj', patterns=[pronoun_aux_verb_adj])
matcher2.add('pronoun_aux_verb_adv_adj', patterns=[pronoun_aux_verb_adv_adj])
matcher2.add('sujeito_verb_aux_adj', patterns=[sujeito_verb_aux_adj])
matcher2.add('sujeito_verb_aux_adv_adj', patterns=[sujeito_verb_aux_adv_adj])
matcher2.add('sujeito_adj', patterns=[sujeito_adj])
matcher2.add('pronoun_verb_adv_adj', patterns=[pronoun_verb_adv_adj])
matcher2.add('pronoun_verb_adj', patterns=[pronoun_verb_adj])
matcher2.add('sujeito_verb_adv_adj', patterns=[sujeito_verb_adv_adj])
matcher2.add('sujeito_verb_adj', patterns=[sujeito_verb_adj])
# matcher2.add('sujeito_aux_adv_verb', patterns=[sujeito_aux_adv_verb])
# matcher2.add('sujeito_aux_verb', patterns=[sujeito_aux_verb])
matcher2.add('sujeito_pron_verb_adj', patterns=[sujeito_pron_verb_adj])
matcher2.add('pronoun_pron_verb_adj', patterns=[pronoun_pron_verb_adj])
matcher2.add('sujeito_pron_verb_adv_adj', patterns=[sujeito_pron_verb_adv_adj])
matcher2.add('pronoun_pron_verb_adv_adj', patterns=[pronoun_pron_verb_adv_adj])
matcher2.add('pronoun_aux_pron_verb_adj', patterns=[pronoun_aux_pron_verb_adj])
matcher2.add('sujeito_aux_pron_verb_adj', patterns=[sujeito_aux_pron_verb_adj])
matcher2.add('pronoun_aux_pron_verb_adv_adj', patterns=[pronoun_aux_pron_verb_adv_adj])
matcher2.add('sujeito_aux_pron_verb_adv_adj', patterns=[sujeito_aux_pron_verb_adv_adj])
matcher2.add('sujeito_verb_pron_verb_adj', patterns=[sujeito_verb_pron_verb_adj])
matcher2.add('pronoun_verb_pron_verb_adj', patterns=[pronoun_verb_pron_verb_adj])
matcher2.add('pronoun_verb_pron_verb_adv_adj', patterns=[pronoun_verb_pron_verb_adv_adj])
matcher2.add('sujeito_verb_pron_verb_adv_adj', patterns=[sujeito_verb_pron_verb_adv_adj])
matcher2.add('nome_aux_adj', patterns=[nome_aux_adj])
matcher2.add('nome_aux_adv_adj', patterns=[nome_aux_adv_adj])
matcher2.add('nome_aux_aux_adj', patterns=[nome_aux_aux_adj])
matcher2.add('nome_aux_aux_adv_adj', patterns=[nome_aux_aux_adv_adj])
matcher2.add('nome_aux_verb_adj', patterns=[nome_aux_verb_adj])
matcher2.add('nome_aux_verb_adv_adj', patterns=[nome_aux_verb_adv_adj])
matcher2.add('nome_verb_aux_adj', patterns=[nome_verb_aux_adj])
matcher2.add('nome_verb_aux_adv_adj', patterns=[nome_verb_aux_adv_adj])
matcher2.add('nome_adj', patterns=[nome_adj])
matcher2.add('nome_verb_adv_adj', patterns=[nome_verb_adv_adj])
matcher2.add('nome_verb_adj', patterns=[nome_verb_adj])
# matcher2.add('nome_aux_adv_verb', patterns=[nome_aux_adv_verb])
matcher2.add('nome_aux_verb', patterns=[nome_aux_verb])
matcher2.add('nome_pron_verb_adj', patterns=[nome_pron_verb_adj])
matcher2.add('nome_pron_verb_adv_adj', patterns=[nome_pron_verb_adv_adj])
matcher2.add('nome_aux_pron_verb_adj', patterns=[nome_aux_pron_verb_adj])
matcher2.add('nome_aux_pron_verb_adv_adj', patterns=[nome_aux_pron_verb_adv_adj])
matcher2.add('nome_verb_pron_verb_adj', patterns=[nome_verb_pron_verb_adj])
matcher2.add('nome_verb_pron_verb_adv_adj', patterns=[nome_verb_pron_verb_adv_adj])


In [ ]:
resultados = []

for index, row in dataframe.iterrows():
    sentenca = str(row['Letra da Música'])
    doc = pln(sentenca)
    results = matcher2(doc, as_spans=True)

    for result in results:
      if len(result) > 0:
        resultados.append({'Padrão': pln.vocab[result.label].text,
                           'Frase': result,
                           'Nome da Música': row['Nome da Música'],
                           'Gênero Musical': row['Gênero Musical'],
                           'Letra da Música': sentenca})
resultados_masculino = pd.DataFrame(resultados)
resultados_masculino.head()

,Padrão,Frase,Nome da Música,Gênero Musical,Letra da Música
0,sujeito_aux_adj,"(ele, era, doutor)",Papel do Mal (part. MC Pedrinho),Funk Carioca,Dizem: traz felicidade E outros dizem que não ...
1,sujeito_adj,"(menino, pescador)",Vida Bela (Praia Branca),Bossa Nova,"Praia branca, tristeza Mar sem fim Lua nova Mu..."
2,sujeito_adj,"(menino, pescador)",Vida Bela (Praia Branca),Bossa Nova,"Praia branca, tristeza Mar sem fim Lua nova Mu..."
3,nome_adj,"(Pedro, pedreiro)",Pedro Pedreiro,Bossa Nova,Pedro pedreiro bem cedo esperando o trem Mãe a...
4,nome_adj,"(Pedro, pedreiro)",Pedro Pedreiro,Bossa Nova,Pedro pedreiro bem cedo esperando o trem Mãe a...


In [ ]:
resultados_masculino['Frase'] = resultados_masculino['Frase'].astype(str)
resultados_masculino['Frase'] = resultados_masculino['Frase'].str.strip()
resultados_masculino['Frase'] = resultados_masculino['Frase'].str.lower()
resultados_masculino['Frase'] = resultados_masculino['Frase'].str.replace(r'\s+', ' ', regex=True)
resultados_masculino = resultados_masculino.drop_duplicates(subset=['Frase', 'Letra da Música','Nome da Música'])
resultados_masculino.head()

,Padrão,Frase,Nome da Música,Gênero Musical,Letra da Música
0,sujeito_aux_adj,ele era doutor,Papel do Mal (part. MC Pedrinho),Funk Carioca,Dizem: traz felicidade E outros dizem que não ...
1,sujeito_adj,menino pescador,Vida Bela (Praia Branca),Bossa Nova,"Praia branca, tristeza Mar sem fim Lua nova Mu..."
2,sujeito_adj,menino pescador,Vida Bela (Praia Branca),Bossa Nova,"Praia branca, tristeza Mar sem fim Lua nova Mu..."
3,nome_adj,pedro pedreiro,Pedro Pedreiro,Bossa Nova,Pedro pedreiro bem cedo esperando o trem Mãe a...
13,nome_adj,shazan capitão,Uau!,Bossa Nova,"Informal, não convencional Atual, não há nada ..."


In [ ]:
resultados_masculino.shape

(227, 5)

In [ ]:
resultados_masculino.to_csv('exp_masc.csv', index=False)
from google.colab import files
files.download('exp_masc.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>